In [1]:
import pandas as pd 
import numpy as np
from pulp import *
path=r'C:\Users\84280\Desktop\lianxi\olympic_data.xlsx'
print(path)
df=pd.read_excel(path,index_col=0)
df.head(10)

C:\Users\84280\Desktop\lianxi\olympic_data.xlsx


,人口(100万人),GDP(10億ドル),金,銀,銅
アメリカ,321.774,14682.739,46,37,38
中国,1376.949,5320.232,26,18,26
イギリス,64.716,2676.520,27,23,17
ロシア,143.457,999.832,19,18,19
ドイツ,80.689,3236.726,17,10,15
フランス,64.395,2361.317,10,18,14
日本,126.573,4780.944,12,8,21
オーストラリア,23.969,975.012,8,11,10
イタリア,59.798,1745.045,8,12,8
カナダ,35.940,1359.773,4,3,15


In [2]:

x=df.iloc[:,0:2]
y=df.iloc[:,2:5]
x,y

(         人口(100万人)  GDP(10億ドル)
 アメリカ       321.774   14682.739
 中国        1376.949    5320.232
 イギリス        64.716    2676.520
 ロシア        143.457     999.832
 ドイツ         80.689    3236.726
 フランス        64.395    2361.317
 日本         126.573    4780.944
 オーストラリア     23.969     975.012
 イタリア        59.798    1745.045
 カナダ         35.940    1359.773,
           金   銀   銅
 アメリカ     46  37  38
 中国       26  18  26
 イギリス     27  23  17
 ロシア      19  18  19
 ドイツ      17  10  15
 フランス     10  18  14
 日本       12   8  21
 オーストラリア   8  11  10
 イタリア      8  12   8
 カナダ       4   3  15)

In [3]:
x1=np.array(x)
y1=np.array(y)
prob=LpProblem(name='efficiency',sense=LpMaximize)
u=[LpVariable('u'+str(j+1),lowBound=0) for j in range(2)]
v=[LpVariable('v'+str(j+1),lowBound=0) for j in range(3)]
u1=np.transpose(np.array(u))
row=x1[0]

rowy=y1[0]
rema=np.delete(x1,0,axis=0).T
remay=np.delete(y1,0,axis=0)
prob+=np.dot(row,u)==1
print(rema)
print(u1)
a=np.dot(rema.T,u1).T
print(a)
print(a[1])



[[1376.949   64.716  143.457   80.689   64.395  126.573   23.969   59.798
    35.94 ]
 [5320.232 2676.52   999.832 3236.726 2361.317 4780.944  975.012 1745.045
  1359.773]]
[u1 u2]
[1376.949*u1 + 5320.232*u2 + 0.0 64.716*u1 + 2676.52*u2 + 0.0
 143.457*u1 + 999.832*u2 + 0.0 80.689*u1 + 3236.726*u2 + 0.0
 64.395*u1 + 2361.317*u2 + 0.0 126.573*u1 + 4780.944*u2 + 0.0
 23.969*u1 + 975.012*u2 + 0.0 59.798*u1 + 1745.045*u2 + 0.0
 35.94*u1 + 1359.773*u2 + 0.0]
64.716*u1 + 2676.52*u2


In [4]:
a=[[123],[22]]
a[0].append(1)
print(a)



[[123, 1], [22]]


In [7]:
def dea(name,n,l,m):
    df = pd.read_excel(name,index_col=0)
    L=list(df.index)
    R=[[] for _ in range(n)]
    X=np.array(df.iloc[:,0:l])
    Y=np.array(df.iloc[:,l:l+m])
    for i in range(n):
        R[i].append(L[i])
        prob=LpProblem(name='efficiency'+str(i+1),sense=LpMaximize)
        u=[LpVariable('u'+str(i+1),lowBound=0) for i in range(l)]
        v=[LpVariable('v'+str(i+1),lowBound=0) for i in range(m)]
        prob+=lpDot(v,Y[i])
        prob+=lpDot(u,X[i])==1
        for j in range(n):
            prob+=lpDot(v,Y[j])<=lpDot(u,X[j])
        prob.solve()
        R[i].append(value(prob.objective))
        us=np.array([value(u[i]) for i in range(l)])
        vs=np.array([value(v[i]) for i in range(m)]) 
        R[i].append(us)
        R[i].append(vs)
        temp=[] 
        for j in range(n):
            if j==i:
                continue 
            if np.abs(np.dot(vs,Y[j])-np.dot(us,X[j]))<=10**(-6):
                temp.append(L[j])
                R[i].append(temp)
    return R 
R1=dea(path,10,2,3)
print(R1)
for i in range(10):
    print(df.index[i],R1[i])

[['アメリカ', 0.3697814572, array([0.00310777, 0.        ]), array([0.0055589, 0.       , 0.0030019]), ['イギリス', 'オーストラリア'], ['イギリス', 'オーストラリア']], ['中国', 0.257167573, array([0.        , 0.00018796]), array([0.        , 0.        , 0.00989106]), ['ロシア']], ['イギリス', 0.9999999970000001, array([0.01545213, 0.        ]), array([0.02763936, 0.        , 0.01492572]), ['オーストラリア']], ['ロシア', 1.0000000009999999, array([0.        , 0.00100017]), array([0.        , 0.        , 0.05263158])], ['ドイツ', 0.56625202, array([0.00294968, 0.00023542]), array([0.02317865, 0.        , 0.011481  ]), ['イギリス', 'ロシア', 'オーストラリア'], ['イギリス', 'ロシア', 'オーストラリア'], ['イギリス', 'ロシア', 'オーストラリア']], ['フランス', 0.666693702, array([0.00209373, 0.00036639]), array([0.        , 0.03703854, 0.        ]), ['ロシア', 'オーストラリア'], ['ロシア', 'オーストラリア']], ['日本', 0.4118923692, array([0.00154842, 0.00016817]), array([0.00216244, 0.        , 0.01837824]), ['ロシア', 'オーストラリア', 'カナダ'], ['ロシア', 'オーストラリア', 'カナダ'], ['ロシア', 'オーストラリア', 'カナダ']], ['オーストラリア', 1.000

In [14]:

    


def DEA(name,n,l,m):
    df=pd.read_excel(name,index_col=0)
    L=list(df.index)
    ##name:path, n:total number, l number of input, m number of output
    R=[[] for i in range(n)]
    print(R)
    x1=np.array(df.iloc[:,0:l]) #input
    y1=np.array(df.iloc[:,l:m+l]) #output

    for i in range(n):
        ## define problem and weight, u for input , v for output 
        prob=LpProblem(name='efficiency'+str(i+1),sense=LpMaximize)
        u=[LpVariable('u'+str(j+1),lowBound=0) for j in range(l)]
        v=[LpVariable('v'+str(j+1),lowBound=0) for j in range(m)]

        prob+=np.dot(v,y1[i])
        for e in range(n):
            if e==i:
                prob+=np.dot(u,x1[e])==1
            else:
                prob+=np.dot(v,y1[e])-np.dot(u,x1[e])<=0
        print(prob)
    ### subject ineq
        prob.solve()
        R[i].append(value(prob.objective))
        us=np.array([value(u[i]) for i in range(l)])
        vs=np.array([value(v[i]) for i in range(m)])
        R[i].append(us)
        R[i].append(vs)
        temp=[] ###  comparing set
        for j in range(n):
            if j == i:
                continue
            if np.abs(np.dot(vs,y1[j])-np.dot(us,x1[j]))<=10**(-6):
                temp.append(L[j])
        R[i].append(temp)
    return R

R1=DEA(path,10,2,3)
for i in range(10):
    print(df.index[i],R1[i])

[[], [], [], [], [], [], [], [], [], []]
efficiency1:
MAXIMIZE
46*v1 + 37*v2 + 38*v3 + 0
SUBJECT TO
_C1: 321.774 u1 + 14682.739 u2 = 1

_C2: - 1376.949 u1 - 5320.232 u2 + 26 v1 + 18 v2 + 26 v3 <= 0

_C3: - 64.716 u1 - 2676.52 u2 + 27 v1 + 23 v2 + 17 v3 <= 0

_C4: - 143.457 u1 - 999.832 u2 + 19 v1 + 18 v2 + 19 v3 <= 0

_C5: - 80.689 u1 - 3236.726 u2 + 17 v1 + 10 v2 + 15 v3 <= 0

_C6: - 64.395 u1 - 2361.317 u2 + 10 v1 + 18 v2 + 14 v3 <= 0

_C7: - 126.573 u1 - 4780.944 u2 + 12 v1 + 8 v2 + 21 v3 <= 0

_C8: - 23.969 u1 - 975.012 u2 + 8 v1 + 11 v2 + 10 v3 <= 0

_C9: - 59.798 u1 - 1745.045 u2 + 8 v1 + 12 v2 + 8 v3 <= 0

_C10: - 35.94 u1 - 1359.773 u2 + 4 v1 + 3 v2 + 15 v3 <= 0

VARIABLES
u1 Continuous
u2 Continuous
v1 Continuous
v2 Continuous
v3 Continuous

efficiency2:
MAXIMIZE
26*v1 + 18*v2 + 26*v3 + 0
SUBJECT TO
_C1: - 321.774 u1 - 14682.739 u2 + 46 v1 + 37 v2 + 38 v3 <= 0

_C2: 1376.949 u1 + 5320.232 u2 = 1

_C3: - 64.716 u1 - 2676.52 u2 + 27 v1 + 23 v2 + 17 v3 <= 0

_C4: - 143.457 u1 - 9

[[], [], [], [], [], [], [], [], [], []]
efficiency1:
MAXIMIZE
46*v1 + 37*v2 + 38*v3 + 0
SUBJECT TO
_C1: 321.774 u1 + 14682.739 u2 = 1

_C2: - 1376.949 u1 - 5320.232 u2 + 26 v1 + 18 v2 + 26 v3 <= 0

_C3: - 64.716 u1 - 2676.52 u2 + 27 v1 + 23 v2 + 17 v3 <= 0

_C4: - 143.457 u1 - 999.832 u2 + 19 v1 + 18 v2 + 19 v3 <= 0

_C5: - 80.689 u1 - 3236.726 u2 + 17 v1 + 10 v2 + 15 v3 <= 0

_C6: - 64.395 u1 - 2361.317 u2 + 10 v1 + 18 v2 + 14 v3 <= 0

_C7: - 126.573 u1 - 4780.944 u2 + 12 v1 + 8 v2 + 21 v3 <= 0

_C8: - 23.969 u1 - 975.012 u2 + 8 v1 + 11 v2 + 10 v3 <= 0

_C9: - 59.798 u1 - 1745.045 u2 + 8 v1 + 12 v2 + 8 v3 <= 0

_C10: - 35.94 u1 - 1359.773 u2 + 4 v1 + 3 v2 + 15 v3 <= 0

VARIABLES
u1 Continuous
u2 Continuous
v1 Continuous
v2 Continuous
v3 Continuous

efficiency2:
MAXIMIZE
26*v1 + 18*v2 + 26*v3 + 0
SUBJECT TO
_C1: - 321.774 u1 - 14682.739 u2 + 46 v1 + 37 v2 + 38 v3 <= 0

_C2: 1376.949 u1 + 5320.232 u2 = 1

_C3: - 64.716 u1 - 2676.52 u2 + 27 v1 + 23 v2 + 17 v3 <= 0

_C4: - 143.457 u1 - 9

In [9]:
print(df.index)

for i in range(10):
    print(df.index[i],R1[i])

Index(['アメリカ', '中国', 'イギリス', 'ロシア', 'ドイツ', 'フランス', '日本', 'オーストラリア', 'イタリア',
       'カナダ'],
      dtype='object')
アメリカ [0.3697814572, array([0.00310777, 0.        ]), array([0.0055589, 0.       , 0.0030019]), ['イギリス', 'オーストラリア']]
中国 [0.257167573, array([0.        , 0.00018796]), array([0.        , 0.        , 0.00989106]), ['ロシア']]
イギリス [1.250005797, array([0.01545213, 0.        ]), array([0.04629651, 0.        , 0.        ]), ['オーストラリア']]
ロシア [2.141852672, array([0.        , 0.00100017]), array([0.07605752, 0.        , 0.03667156]), ['イギリス', 'オーストラリア']]
ドイツ [0.56625202, array([0.00294968, 0.00023542]), array([0.02317865, 0.        , 0.011481  ]), ['イギリス', 'ロシア', 'オーストラリア']]
フランス [0.666693702, array([0.00209373, 0.00036639]), array([0.        , 0.03703854, 0.        ]), ['ロシア', 'オーストラリア']]
日本 [0.4118923692, array([0.00154842, 0.00016817]), array([0.00216244, 0.        , 0.01837824]), ['ロシア', 'オーストラリア', 'カナダ']]
オーストラリア [1.462759557, array([0.0133688 , 0.00069698]), array([0.        , 0.0